# Taxonomic Annotation of Reads

This notebook will go through the workflow assigning taxonomy to reads from a microbiome

Step 1: Assign taxonomy to reads (using 3 parts that run in a single job)
1. Taxonomic assignment of reads using Kraken2
2. Refinement of the taxonomic annotation using Bracken
3. Separation of human and non-human reads using KrakenTools

Step 2: Assign taxonomy to contigs from megahit (with 3 parts as above)

Step 3: Assign taxonomy to contigs from metaspades (with 3 parts as above)

## Getting Started

You will need to rerun this section each time you come back to this notebook to reset all directories and variables.

In [ ]:
# set the variables for your netid and xfile
netid = "MY_NETID"
xfile = "MY_XFILE"

In [ ]:
# Go into the working directory
work_dir = "/xdisk/bhurwitz/bh_class/" + netid + "/assignments/11_taxonomy"
%cd $work_dir

## Creating a config file
Let's create a config file with all of the variables we will need in the scripts below. Then when we want to use these variables in the script, we will "source" the config file to set the variables.

In [ ]:
# create a config file with all of the variables you need
# notice that now we are defining the tools here too, to make out scripts easier to read below
# also, if we want to update to a newer version of the tool we can just edit here.
!echo "export NETID=$netid" > config.sh
!echo "export XFILE=$xfile" >> config.sh
!echo "export WORK_DIR=/xdisk/bhurwitz/bh_class/$netid/assignments/11_taxonomy" >> config.sh
!echo "export XFILE_DIR=/xdisk/bhurwitz/bh_class/$netid/assignments/05_getting_data" >> config.sh
!echo "export FASTQ_DIR=/xdisk/bhurwitz/bh_class/$netid/assignments/07_contam_removal" >> config.sh
!echo "export MEGAHIT_CONTIG_DIR=/xdisk/bhurwitz/bh_class/$netid/assignments/08_assembly/out_megahit" >> config.sh
!echo "export METASPADES_CONTIG_DIR=/xdisk/bhurwitz/bh_class/$netid/assignments/08_assembly/out_spades" >> config.sh
!echo "export CONTAINERS=/contrib/singularity/shared/bhurwitz" >> config.sh
!echo "export KRAKEN2=/contrib/singularity/shared/bhurwitz/kraken2:2.1.3--pl5321hdcf5f25_0.sif" >> config.sh
!echo "export BRACKEN=/contrib/singularity/shared/bhurwitz/bracken:2.8--py39h1f90b4d_1.sif" >> config.sh
!echo "export KRAKENTOOLS=/contrib/singularity/shared/bhurwitz/krakentools:1.2--pyh5e36f6f_0.sif" >> config.sh
!echo "export DB_DIR=/groups/bhurwitz/databases/kraken2/k2_pluspf_20230605" >> config.sh
!echo "export KMER_SIZE=100" >> config.sh


In [ ]:
# check the config file to be sure it is correct
# Is your netid and xfile correct? Do you have the right directories?
!cat config.sh

## Step 1: Assigning taxonomy to the reads

In this step, we will assign taxonomy to each of the reads in our microbiome (where possible). This script involves multiple steps in the annotation process including:

1. Running Kraken2 to assign reads to organisms by taxonomic rank. 
2. Running Bracken a tool that refines the Kraken2 output to try to reassign reads to higher species-level taxonomy
3. Separating out the reads based on their taxonomy using KrakenTools. We do this at a high level human / non-human.

The final part in this analysis looks for any remaining human contamination in the files, after we align to the human genome using bowtie2. Do you see any reads reported as human? Try "Blasting" these to see if they are indeed human reads that we missed by the alignment. 



In [ ]:
# Create a script to run Kraken, Bracken, and KrakenTools to assign taxonomy at the read-level.
# A few important points:
# 1. We are using the variables from the config file via the `source ./config.sh` command in the script.
# 2. Kraken2 runs on each of the fastq files in the trimmed/human screened $FASTQ_DIR
# 3. The results will be written into our $WORK_DIR
# 4. Notice that we are asking for alot more resource (24 cores and 5G of memory per core), we are also asking for 10 hours of runtime.
# But, runtime will likely be much less.
my_code = '''#!/bin/bash
#SBATCH --ntasks=1
#SBATCH --nodes=1             
#SBATCH --time=10:00:00   
#SBATCH --partition=standard
#SBATCH --account=bhurwitz
#SBATCH --array=0-4                         
#SBATCH --output=Job-read-taxonomy-%a.out
#SBATCH --cpus-per-task=24
#SBATCH --mem-per-cpu=5G  

pwd; hostname; date

source $SLURM_SUBMIT_DIR/config.sh
names=($(cat $XFILE_DIR/$XFILE))

SAMPLE_ID=${names[${SLURM_ARRAY_TASK_ID}]}

### reads with human removed to match to the reference database
PAIR1=${FASTQ_DIR}/${SAMPLE_ID}_1.fastq.gz
PAIR2=${FASTQ_DIR}/${SAMPLE_ID}_2.fastq.gz

KRAKEN_OUTDIR=${WORK_DIR}/out_reads_taxonomy
OUTDIR=${KRAKEN_OUTDIR}/${SAMPLE_ID}
HUMAN_READ_DIR=${OUTDIR}/human_reads
NONHUMAN_READ_DIR=${OUTDIR}/nonhuman_reads

### create the outdir if it does not exist
if [[ ! -d "$KRAKEN_OUTDIR" ]]; then
  echo "$KRAKEN_OUTDIR does not exist. Directory created"
  mkdir $KRAKEN_OUTDIR
fi

if [[ ! -d "$OUTDIR" ]]; then
  echo "$OUTDIR does not exist. Directory created"
  mkdir $OUTDIR
fi

if [[ ! -d "$HUMAN_READ_DIR" ]]; then
  echo "$HUMAN_READ_DIR does not exist. Directory created"
  mkdir $HUMAN_READ_DIR
fi

if [[ ! -d "$NONHUMAN_READ_DIR" ]]; then
  echo "$NONHUMAN_READ_DIR does not exist. Directory created"
  mkdir $NONHUMAN_READ_DIR
fi

# check input
echo ${PAIR1}
echo ${PAIR2}
echo ${OUTDIR}

apptainer run ${KRAKEN2} kraken2 --db ${DB_DIR} --paired \
  --classified-out ${OUTDIR}/cseqs#.fq --output ${OUTDIR}/kraken_results.txt \
  --report ${OUTDIR}/kraken_report.txt --use-names --threads ${SLURM_CPUS_PER_TASK} \
  ${PAIR1} ${PAIR2}

# refine hits with Bracken
REPORT="${OUTDIR}/kraken_report.txt"
RESULTS="${OUTDIR}/kraken_results.txt"
apptainer run ${BRACKEN} est_abundance.py -i ${REPORT} -o ${OUTDIR}/bracken_results.txt -k ${DB_DIR}/database${KMER_SIZE}mers.kmer_distrib

# get human and non-human reads (microbial)
TAXID=9606
HUMAN_R1="${HUMAN_READ_DIR}/r1.fq"
HUMAN_R2="${HUMAN_READ_DIR}/r2.fq"

BRACKEN_REPORT="${OUTDIR}/kraken_report_bracken_species.txt"
BRACKEN_RESULTS="${OUTDIR}/bracken_results.txt"

apptainer run ${KRAKENTOOLS} extract_kraken_reads.py -k ${RESULTS} \
 -r ${BRACKEN_REPORT} -s1 ${PAIR1} -s2 ${PAIR2} --taxid ${TAXID} \
 -o ${HUMAN_R1} -o2 ${HUMAN_R2} --include-children --fastq-output 

gzip ${HUMAN_READ_DIR}/r1.fq
gzip ${HUMAN_READ_DIR}/r2.fq

### selects all reads NOT from a given set of Kraken taxids (and all children)

NONHUMAN_R1="${NONHUMAN_READ_DIR}/r1.fq"
NONHUMAN_R2="${NONHUMAN_READ_DIR}/r2.fq"

apptainer run ${KRAKENTOOLS} extract_kraken_reads.py -k ${RESULTS} \
 -r ${BRACKEN_REPORT} -s1 ${PAIR1} -s2 ${PAIR2} --taxid ${TAXID} \
 -o ${NONHUMAN_R1} -o2 ${NONHUMAN_R2} --include-children \
 --exclude --fastq-output 

gzip ${NONHUMAN_READ_DIR}/r1.fq
gzip ${NONHUMAN_READ_DIR}/r2.fq

echo "Finished `date`"

'''

with open('run_read_taxonomy_parallel.sh', mode='w') as file:
    file.write(my_code)

In [ ]:
# Check the code and make sure your script above was created.
!cat run_read_taxonomy_parallel.sh

In [ ]:
# you should be in your working directory when you run this script
# do you see your config.sh file, and the run_kraken_parallel.sh script?
!pwd
!ls

In [ ]:
# Let's run sbatch to run kraken2, bracken, and krakentools to assign taxonomy to our reads
# Remember that this may take a while to run (~2 hours), so take a break, and get a coffee.
!sbatch ./run_read_taxonomy_parallel.sh

In [ ]:
# Welcome back
# You can check if it is running using the squeue command
# Check for all jobs under your netid
!squeue --user=$netid

In [ ]:
# You can check to see if there are any errors by looking at one of the job output files
# Remember that this is going to give you a ton of data! These scripts report a lot in the log.
# You can look at Job-rem_human-0.out
!ls
!tail Job-read-taxonomy-0.out

Nice, you should now have all of the reads assigned to different organisms. We are going to use this in our next steps to produce graphs and examine the taxonomic content of our samples.

Let's take a quick peak to see if we were able to remove any additional human in the samples. Remember that this was supposed to be gone! But, alignment algorithms aren't perfect. 

In [ ]:
# Are the r1.fq.gz and r2.fq.gz files in your sample directories greater than zero in size?
# Mine are, so likely you have some human contamination remaining.
!ls -l $work_dir/out_reads_taxonomy/ERR*/human_reads

In [ ]:
# Fear not, we removed all of the human contamination using krakentools to get all of the reads that don't match human
# You can find them in the nonhuman_reads directory. 
!ls -l $work_dir/out_reads_taxonomy/ERR*/nonhuman_reads

In [ ]:
# You can also check some of these reads to see if they are actually human
# Let's convert the first sequence in each of the files into fasta format
!for dir in `ls ./out_reads_taxonomy`; do gzip -dc ./out_reads_taxonomy/$dir/human_reads/r1.fq.gz | head -2 | sed 's/^@/>/'; done

Now, copy and paste the 5 sequences from above and use the online BLAST tool to see if they are human.  To do this, 

1. Go to: https://blast.ncbi.nlm.nih.gov/Blast.cgi. 
2. Click on the Nucleotide BLAST button
3. Paste your 5 fasta formatted sequences into the "Enter Query Sequence" box.
4. Click on the "BLAST" button on the bottom of the page (which will use all defaults)

What do you get? I found that most of my sequences had "no significant match", and one actually matched a bacterial genome. So, these reads may be less significant matches to the human genome (remember that we are using k-mers to find them and not entire sequences). They may also be matching to regions of the human genome that are known to be microbial in origin:

[Microbial Genes in the Human Genome: Lateral Transfer or Gene Loss?](https://www.science.org/doi/10.1126/science.1061036)

Or, oppositely, we can find human sequences in bacterial genomes:

[Human contamination in bacterial genomes has created thousands of spurious proteins](https://genome.cshlp.org/content/29/6/954)

Ugh, just when we thought we were getting somewhere!

## Step 2: Assigning taxonomy to the contigs from Megahit

In this step, we will assign taxonomy to each of the contigs from our megahit assembly (where possible). This script involves multiple steps in the annotation process (just as before for the reads) including:

1. Running Kraken2 to assign contigs to organisms by taxonomic rank. 
2. Running Bracken a tool that refines the Kraken2 output to try to reassign contigs to higher species-level taxonomy
3. Separating out the contigs based on their taxonomy using KrakenTools. We do this at a high level human / non-human.

In [ ]:
# Create a script to run Kraken, Bracken, and KrakenTools to assign taxonomy at the contig-level for megahit.
# A few important points:
# 1. We are using the variables from the config file via the `source ./config.sh` command in the script.
# 2. Kraken2 runs on the final.contigs.fa file in the contig directory
# 3. The results will be written into our $WORK_DIR
# 4. Notice that we are asking for alot more resource (24 cores and 5G of memory per core), we are also asking for 10 hours of runtime.
# But, runtime will likely be much less.
my_code = '''#!/bin/bash
#SBATCH --ntasks=1
#SBATCH --nodes=1             
#SBATCH --time=10:00:00   
#SBATCH --partition=standard
#SBATCH --account=bhurwitz
#SBATCH --array=0-4                         
#SBATCH --output=Job-megahit-taxonomy-%a.out
#SBATCH --cpus-per-task=24
#SBATCH --mem-per-cpu=5G  

pwd; hostname; date

source $SLURM_SUBMIT_DIR/config.sh
names=($(cat $XFILE_DIR/$XFILE))

SAMPLE_ID=${names[${SLURM_ARRAY_TASK_ID}]}

### contigs
CONTIGS=${MEGAHIT_CONTIG_DIR}/${SAMPLE_ID}/final.contigs.fa

KRAKEN_OUTDIR=${WORK_DIR}/out_megahit_taxonomy
OUTDIR=${KRAKEN_OUTDIR}/${SAMPLE_ID}
HUMAN_READ_DIR=${OUTDIR}/human_contigs
NONHUMAN_READ_DIR=${OUTDIR}/nonhuman_contigs

### create the outdir if it does not exist
if [[ ! -d "$KRAKEN_OUTDIR" ]]; then
  echo "$KRAKEN_OUTDIR does not exist. Directory created"
  mkdir $KRAKEN_OUTDIR
fi
                                  
if [[ ! -d "$OUTDIR" ]]; then
  echo "$OUTDIR does not exist. Directory created"
  mkdir $OUTDIR
fi

if [[ ! -d "$HUMAN_READ_DIR" ]]; then
  echo "$HUMAN_READ_DIR does not exist. Directory created"
  mkdir $HUMAN_READ_DIR
fi

if [[ ! -d "$NONHUMAN_READ_DIR" ]]; then
  echo "$NONHUMAN_READ_DIR does not exist. Directory created"
  mkdir $NONHUMAN_READ_DIR
fi

apptainer run ${KRAKEN2} kraken2 --db ${DB_DIR} --classified-out \
 ${OUTDIR}/cseqs#.fa --output ${OUTDIR}/kraken_results.txt \
 --report ${OUTDIR}/kraken_report.txt --use-names --threads ${SLURM_CPUS_PER_TASK} \
 ${CONTIGS}

# refine hits with Bracken
REPORT="${OUTDIR}/kraken_report.txt"
RESULTS="${OUTDIR}/kraken_results.txt"
apptainer run ${BRACKEN} est_abundance.py -i ${REPORT} -o ${OUTDIR}/bracken_results.txt -k ${DB_DIR}/database${KMER_SIZE}mers.kmer_distrib

# get human and non-human reads (microbial)
TAXID=9606
HUMAN_R1="${HUMAN_READ_DIR}/contigs.fa"

BRACKEN_REPORT="${OUTDIR}/kraken_report_bracken_species.txt"
BRACKEN_RESULTS="${OUTDIR}/bracken_results.txt"

apptainer run ${KRAKENTOOLS} extract_kraken_reads.py -k ${RESULTS} \
 -r ${BRACKEN_REPORT} -s1 ${CONTIGS} --taxid ${TAXID} -o ${HUMAN_R1} \
 --include-children 

if [[ -f "${HUMAN_READ_DIR}/contigs.fa" ]]; then
    gzip ${HUMAN_READ_DIR}/contigs.fa
fi

### selects all reads NOT from a given set of Kraken taxids (and all children)

NONHUMAN_R1="${NONHUMAN_READ_DIR}/contigs.fa"

apptainer run ${KRAKENTOOLS} extract_kraken_reads.py -k ${RESULTS} \
 -r ${BRACKEN_REPORT} -s1 ${CONTIGS} --taxid ${TAXID} -o ${NONHUMAN_R1} \
 --include-children --exclude 

if [[ -f "${NONHUMAN_READ_DIR}/contigs.fa" ]]; then
    gzip ${NONHUMAN_READ_DIR}/contigs.fa
fi

echo "Finished `date`"

'''

with open('run_megahit_taxonomy_parallel.sh', mode='w') as file:
    file.write(my_code)

In [ ]:
# Let's kick this off!
!sbatch ./run_megahit_taxonomy_parallel.sh

In [ ]:
# Check back later to see if you were able to get human contigs. Are the file sizes > 0?
!ls -l $work_dir/out_megahit_taxonomy/ERR*/human_contigs

In [ ]:
# Do you see contigs assigned to non-human organisms? (aka microbes)
!ls -l $work_dir/out_megahit_taxonomy/ERR*/nonhuman_contigs

## Step 3: Assigning taxonomy to the contigs from Metaspades

In this step, we will assign taxonomy to each of the contigs from our metaspades assembly (where possible). This script involves multiple steps in the annotation process (just as before for the reads) including:

1. Running Kraken2 to assign contigs to organisms by taxonomic rank. 
2. Running Bracken a tool that refines the Kraken2 output to try to reassign contigs to higher species-level taxonomy
3. Separating out the contigs based on their taxonomy using KrakenTools. We do this at a high level human / non-human.

In [ ]:
# Create a script to run Kraken, Bracken, and KrakenTools to assign taxonomy at the contig-level for metaspades.
# A few important points:
# 1. We are using the variables from the config file via the `source ./config.sh` command in the script.
# 2. Kraken2 runs on the final.contigs.fa file in the contig directory
# 3. The results will be written into our $WORK_DIR
# 4. Notice that we are asking for alot more resource (24 cores and 5G of memory per core), we are also asking for 10 hours of runtime.
# But, runtime will likely be much less.
my_code = '''#!/bin/bash
#SBATCH --ntasks=1
#SBATCH --nodes=1             
#SBATCH --time=10:00:00   
#SBATCH --partition=standard
#SBATCH --account=bhurwitz
#SBATCH --array=0-4                         
#SBATCH --output=Job-metaspades-taxonomy-%a.out
#SBATCH --cpus-per-task=24
#SBATCH --mem-per-cpu=5G  

pwd; hostname; date

source $SLURM_SUBMIT_DIR/config.sh
names=($(cat $XFILE_DIR/$XFILE))

SAMPLE_ID=${names[${SLURM_ARRAY_TASK_ID}]}

### contigs
CONTIGS=${METASPADES_CONTIG_DIR}/${SAMPLE_ID}/contigs.fasta

KRAKEN_OUTDIR=${WORK_DIR}/out_metaspades_taxonomy
OUTDIR=${KRAKEN_OUTDIR}/${SAMPLE_ID}
HUMAN_READ_DIR=${OUTDIR}/human_contigs
NONHUMAN_READ_DIR=${OUTDIR}/nonhuman_contigs

### create the outdir if it does not exist
if [[ ! -d "$KRAKEN_OUTDIR" ]]; then
  echo "$KRAKEN_OUTDIR does not exist. Directory created"
  mkdir $KRAKEN_OUTDIR
fi
                                  
if [[ ! -d "$OUTDIR" ]]; then
  echo "$OUTDIR does not exist. Directory created"
  mkdir $OUTDIR
fi

if [[ ! -d "$HUMAN_READ_DIR" ]]; then
  echo "$HUMAN_READ_DIR does not exist. Directory created"
  mkdir $HUMAN_READ_DIR
fi

if [[ ! -d "$NONHUMAN_READ_DIR" ]]; then
  echo "$NONHUMAN_READ_DIR does not exist. Directory created"
  mkdir $NONHUMAN_READ_DIR
fi

apptainer run ${KRAKEN2} kraken2 --db ${DB_DIR} --classified-out \
 ${OUTDIR}/cseqs#.fa --output ${OUTDIR}/kraken_results.txt \
 --report ${OUTDIR}/kraken_report.txt --use-names --threads ${SLURM_CPUS_PER_TASK} \
 ${CONTIGS}

# refine hits with Bracken
REPORT="${OUTDIR}/kraken_report.txt"
RESULTS="${OUTDIR}/kraken_results.txt"
apptainer run ${BRACKEN} est_abundance.py -i ${REPORT} -o ${OUTDIR}/bracken_results.txt -k ${DB_DIR}/database${KMER_SIZE}mers.kmer_distrib

# get human and non-human reads (microbial)
TAXID=9606
HUMAN_R1="${HUMAN_READ_DIR}/contigs.fa"

BRACKEN_REPORT="${OUTDIR}/kraken_report_bracken_species.txt"
BRACKEN_RESULTS="${OUTDIR}/bracken_results.txt"

apptainer run ${KRAKENTOOLS} extract_kraken_reads.py -k ${RESULTS} \
 -r ${BRACKEN_REPORT} -s1 ${CONTIGS} --taxid ${TAXID} -o ${HUMAN_R1} \
 --include-children

if [[ -f "${HUMAN_READ_DIR}/contigs.fa" ]]; then
    gzip ${HUMAN_READ_DIR}/contigs.fa
fi

### selects all reads NOT from a given set of Kraken taxids (and all children)

NONHUMAN_R1="${NONHUMAN_READ_DIR}/contigs.fa"

apptainer run ${KRAKENTOOLS} extract_kraken_reads.py -k ${RESULTS} \
 -r ${BRACKEN_REPORT} -s1 ${CONTIGS} --taxid ${TAXID} -o ${NONHUMAN_R1} \
 --include-children --exclude

if [[ -f "${NONHUMAN_READ_DIR}/contigs.fa" ]]; then
    gzip ${NONHUMAN_READ_DIR}/contigs.fa
fi

echo "Finished `date`"

'''

with open('run_metaspades_taxonomy_parallel.sh', mode='w') as file:
    file.write(my_code)

In [ ]:
# kick off the script
!sbatch ./run_metaspades_taxonomy_parallel.sh

In [ ]:
# Check back later to see if you were able to get human contigs. Are the file sizes > 0?
!ls -l $work_dir/out_metaspades_taxonomy/ERR*/human_contigs

In [ ]:
# What about microbial contigs?
!ls -l $work_dir/out_metaspades_taxonomy/ERR*/nonhuman_contigs

Great job! We will compare and contrast the taxonomic annotation for the assemblies and the reads in the next homework assignment.

## Final Step
Copy your notebook to the current working directory

In [ ]:
cp ~/11_taxonomy.ipynb $work_dir